In [3]:
def query(self, n):
        unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
        probs = self.predict_prob(unlabeled_data)
        probs_sorted, idxs = probs.sort(descending=True)
        uncertainties = probs_sorted[:, 0] - probs_sorted[:,1]
        return unlabeled_idxs[uncertainties.sort()[1][:n]]

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
iris=pd.read_csv("iris.csv")
train, test = train_test_split(iris, test_size = 0.8)
train_X = train[['sepal.length', 'sepal.width', 'petal.length',
                 'petal.width']]
train_y = train.variety

test_X = test[['sepal.length', 'sepal.width', 'petal.length',
                 'petal.width']]
test_y = test.variety
n_pool = len(train_X)
num = 10
labeled_idxs = np.zeros(n_pool, dtype=bool)
tmp_idxs = np.arange(n_pool)
np.random.shuffle(tmp_idxs)
labeled_idxs[tmp_idxs[:num]] = True
print(len(iris))


150


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
def train(train_X, train_y):
    model = LogisticRegression()
    model.fit(train_X, train_y)
    prediction = model.predict(test_X)
    print('Accuracy:',metrics.accuracy_score(prediction,test_y))

In [ ]:
dataset = get_dataset(args.dataset_name)                   # load dataset
net = get_net(args.dataset_name, device)                   # load network
strategy = get_strategy(args.strategy_name)(dataset, net)  # load strategy

In [ ]:
print("Round 0")
train()
preds = strategy.predict(dataset.get_test_data())
print(f"Round 0 testing accuracy: {dataset.cal_test_acc(preds)}")

for rd in range(1, args.n_round+1):
    print(f"Round {rd}")

    # query
    query_idxs = query(args.n_query)

    # update labels
    update(query_idxs)
    train()

    # calculate accuracy
    preds = predict(dataset.get_test_data())
    print(f"Round {rd} testing accuracy: {dataset.cal_test_acc(preds)}")
